# Playground: RAG with Anthropic Claude 3 Models via Bedrock & Elasticsearch Python Client
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/playground/bedrock-anthropic-elasticsearch-client.ipynb)



In [4]:
!pip install -qU elasticsearch "anthropic[bedrock]"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.6.1 requires build<0.11.0,>=0.10.0, but you have build 1.0.3 which is incompatible.
poetry 1.6.1 requires jsonschema<4.18.0,>=4.10.0, but you have jsonschema 4.19.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


## Example Integration code
To adapt this example for your use-case:
- Update your connection details in the `es_client`
- Replace the es_query with the query suggested in Playground
- Replace the index_source_fields with the fields you want to use for context, per index. 
- Update the prompt to reflect the Plaground prompt

In [2]:
from elasticsearch import Elasticsearch
from anthropic import AnthropicBedrock

from getpass import getpass

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Update the Elasticsearch client with your own Elasticsearch instance
es_client = Elasticsearch(
    "http://localhost:9200",
    api_key=ELASTIC_API_KEY,
)

completion_client = AnthropicBedrock(
    # Authenticate by either providing the keys below or use the default AWS credential providers, such as
    # using ~/.aws/credentials or the "AWS_SECRET_ACCESS_KEY" and "AWS_ACCESS_KEY_ID" environment variables.
    aws_access_key="AWS_ACCESS_KEY_ID",
    aws_secret_key="AWS_SECRET_ACCESS_KEY",
    aws_region="us-west-2",
)

# Update the source fields based on your context field options
index_source_fields = {"bm25-index": ["text"]}


def get_elasticsearch_results(query):
    # Update the query to match your retriever provided in Playground
    es_query = {
        "retriever": {
            "standard": {"query": {"multi_match": {"query": query, "fields": ["text"]}}}
        },
        "size": 3,
    }

    result = es_client.search(index="bm25-index", body=es_query)
    return result["hits"]["hits"]


def create_prompt(question, results):
    context = ""
    for hit in results:
        inner_hit_path = f"{hit['_index']}.{index_source_fields.get(hit['_index'])[0]}"

        ## For semantic_text matches, we need to extract the text from the inner_hits
        if "inner_hits" in hit and inner_hit_path in hit["inner_hits"]:
            context += "\n --- \n".join(
                inner_hit["_source"]["text"]
                for inner_hit in hit["inner_hits"][inner_hit_path]["hits"]["hits"]
            )
        else:
            source_field = index_source_fields.get(hit["_index"])[0]
            hit_context = hit["_source"][source_field]
            context += f"{hit_context}\n"

    # Update the prompt based on your own requirements
    prompt = f"""
  Instructions:
  
  - You are an assistant for question-answering tasks.
  - Answer questions truthfully and factually using only the information presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer!
  - You must always cite the document where the answer was extracted using inline academic citation style [], using the position.
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.
  

  Context:
  {context}

  Question: {question}
  Answer:
  """

    return prompt


def generate_completion(user_prompt):
    response = completion_client.messages.create(
        model="anthropic.claude-3-sonnet-20240229-v1:0",
        messages=[
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=256,
    )

    return response.content


if __name__ == "__main__":
    question = "what is this?"
    elasticsearch_results = get_elasticsearch_results(question)
    context_prompt = create_prompt(question, elasticsearch_results)
    completion = generate_completion(context_prompt)
    print(completion[0].text)

This document appears to be a set of company policies and guidelines related to:

1. A full-time work-from-home policy [Position 1-31]
2. A vacation policy [Position 32-56]
3. A career leveling matrix for software engineers [Position 57-end]

The work-from-home policy outlines the purpose, scope, eligibility, equipment/resources, workspace requirements, communication expectations, work hours, performance standards, time tracking, confidentiality, and health/well-being considerations for employees working remotely full-time.

The vacation policy covers vacation accrual rates, scheduling procedures, vacation pay calculations, handling unplanned absences, and payouts for unused vacation time upon termination.

The career leveling matrix defines the roles, responsibilities, and required skills/experience for three levels of software engineering positions: Junior Software Engineer, Senior Software Engineer, and Principal Software Engineer. It serves as a framework for employee development, 